In [1]:
def update_elo(rating_a, rating_b, result_a, K=32):
    expected_a = 1 / (1 + 10 ** ((rating_b - rating_a) / 400))
    new_rating_a = rating_a + K * (result_a - expected_a)
    return new_rating_a

In [11]:
# 対戦結果に応じた Elo レーティングの更新
def update_each_elo_ratings(trainer_a_rating, trainer_b_rating, is_winner_a: bool):
    if is_winner_a:
        result_a, result_b = 1, 0
    else:
        result_a, result_b = 0, 1

    # ここで一旦、更新前のレーティングを変数に保持する
    rating_a_old = trainer_a_rating
    rating_b_old = trainer_b_rating

    trainer_a_rating = update_elo(rating_a_old, rating_b_old, result_a)
    trainer_b_rating = update_elo(rating_b_old, rating_a_old, result_b)

    return trainer_a_rating, trainer_b_rating

In [17]:
import pandas as pd

df = pd.read_csv("../data/battle_history_修正前.csv")

# レーティング初期化
ratings = {}

# 正しいレーティングを計算
corrected_ratings = []
for _, row in df.iterrows():
    a_name, b_name = row["trainer_a_name"], row["trainer_b_name"]
    winner = row["winner_name"]
    
    # 両トレーナーの初期レーティングを設定（初回は1500）
    a_rating = ratings.get(a_name, 1500)
    b_rating = ratings.get(b_name, 1500)

    # Elo更新
    a_rating_new, b_rating_new = update_each_elo_ratings(a_rating, b_rating, winner == a_name)

    # 更新後のレーティングを記録
    ratings[a_name] = a_rating_new
    ratings[b_name] = b_rating_new

    corrected_ratings.append((a_rating_new, b_rating_new))

df["corrected_trainer_a_rating"], df["corrected_trainer_b_rating"] = zip(*corrected_ratings)

In [27]:
df.query("trainer_a_name == '23_俺出現神呆然'")["trainer_a_rating"]

50    1516
51    1499
52    1515
53    1531
54    1544
55    1528
56    1543
57    1557
58    1571
59    1583
Name: trainer_a_rating, dtype: int64

In [25]:
df.query("trainer_a_name == '23_俺出現神呆然'")["corrected_trainer_a_rating"]

50    1516.000000
51    1499.263693
52    1515.297601
53    1530.677654
54    1544.454316
55    1527.847157
56    1542.780261
57    1556.820054
58    1570.226475
59    1582.395051
Name: corrected_trainer_a_rating, dtype: float64

In [28]:
df.query("trainer_b_name == '23_俺出現神呆然'")["trainer_b_rating"]

121    1595
145    1607
240    1515
305    1515
334    1497
Name: trainer_b_rating, dtype: int64

In [26]:
df.query("trainer_b_name == '23_俺出現神呆然'")["corrected_trainer_b_rating"]

121    1595.237881
145    1607.624824
240    1617.670685
305    1627.317263
334    1604.281266
Name: corrected_trainer_b_rating, dtype: float64

In [29]:
df.columns

Index(['id', 'trainer_a_name', 'trainer_b_name', 'trainer_a_rating',
       'trainer_b_rating', 'log_saved_time', 'winner_name',
       'corrected_trainer_a_rating', 'corrected_trainer_b_rating'],
      dtype='object')

In [31]:
# 修正したもので csv 書き出し
df2 = df[["id", "trainer_a_name", "trainer_b_name", "corrected_trainer_a_rating", "corrected_trainer_b_rating", 'log_saved_time', 'winner_name']]
df2.columns = ["id", "trainer_a_name", "trainer_b_name", "trainer_a_rating", "trainer_b_rating", 'log_saved_time', 'winner_name']
df2.to_csv("../data/battle_history_修正後.csv", index=False, encoding="utf-8-sig")

In [35]:
# レーティングも csv で書き出す

ratings_df = pd.DataFrame(ratings.items(), columns=["trainer_name", "rating"])
# trainer_name は rank_name
# これを id, rank, name, sim_rating にする
ratings_df["rank"] = ratings_df["trainer_name"].apply(lambda x: x.split("_")[0])
ratings_df["name"] = ratings_df["trainer_name"].apply(lambda x: x.split("_")[1])

ratings_df2 = ratings_df[["rank", "name", "rating"]]
ratings_df2.columns = ["rank", "name", "sim_rating"]
ratings_df2["rank"] = ratings_df2["rank"].astype(int)
ratings_df2 = ratings_df2.sort_values(by="rank")
ratings_df2 = ratings_df2.reset_index(drop=True)
ratings_df2.to_csv("../data/rating_修正後.csv", index=False, encoding="utf-8")

/var/folders/cn/l_vdrfmd0pxbx3p33nvb68wr0000gn/T/ipykernel_94342/1108842599.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratings_df2["rank"] = ratings_df2["rank"].astype(int)
